In [ ]:
import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
from tqdm import tqdm
import glob as glob

#thi data contants
from cprl_functions.defined_functions import *
from cprl_functions.state_capture import *
from cprl_functions.text_printing import bordered
from cprl_functions.data_packet_defs import *



In [ ]:
import pull_data
files = pull_data.get_data_files()
print(files)

In [ ]:
print_directory_tree(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Policy Team\Data Packets\k12\data')

In [ ]:
from pathlib import Path

directory = Path('.')
for file in directory.iterdir():
    print(file.name)


In [ ]:

data_directory = Path(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Policy Team\Data Packets\k12\data')

for file in data_directory.iterdir():
    print(file.name)
    if 'naep' in file.name.lower():
        naep_file = file.name
        break
    elif 'data_collection' in file.name.lower():
        data_collection_file = file



# with open(data_directory / naep_file, 'r') as file:
#     excel_file = pd.ExcelFile(file,engine='calamine')
#     for sheet in excel_file.sheet_names:
#         if '- 2024' in sheet:
#             print(sheet) 

excel_file = pd.ExcelFile(data_directory / data_collection_file, engine='calamine')
for sheet in excel_file.sheet_names:
    print(sheet)
    if 'merge' in sheet.lower():
        merge_tags = excel_file.parse(sheet_name=sheet)
        break

print(merge_tags.to_string())



# glob_pat = os.path.join(filepath, 'NAEP*.xlsx')
# files = glob.glob(glob_pat)
# print(merge_tags)
# for file in filepath:
# pd.ExcelFile()

In [ ]:
def clean_str(str):
    cl = str.lower().replace(',','').replace('(','').replace(')','').replace("-","").replace(" ","_").strip()
    return cl

In [ ]:
state_ref

In [ ]:
def get_state_abv(text):
    pat  = re.compile('|'.join(state_list), re.IGNORECASE)
    match = re.search(pat, text)
    if 'District' in match.group(0):
        return 'DC'
        
    string = match.group(0)
    abv = state_ref.get(string)
    return abv


In [ ]:

state_coding

In [ ]:
def get_state_code(state_abv):
    result = state_coding.get(state_abv)
    # print(state_abv)
    # print(result)
    return str(result)

In [ ]:
def pop_column(df, label):
    popped_col = df.pop(label)
    df.insert(0,label,popped_col)
    return df


In [ ]:
# clean up
df = pd.read_csv("data_collection_links.csv")
df['state_abv'] =  df['State'].apply(get_state_abv)
st_abv_pop = df.pop('state_abv')
df.insert(0,'state_abv',st_abv_pop)

df.columns = [clean_str(x) for x in df.columns]
mapping = {}
for i,col in enumerate(df.columns):
    if i > 1:
        mapping[col] = f'metric_{col}'

df = df.rename(columns=mapping)
df['state_code'] = df['state_abv'].apply(get_state_code)
new_df = pop_column(df,'state_code')


In [ ]:
new_df.to_clipboard()

In [ ]:
# counting unique sources
from collections import Counter
statess_dicts = {}
for row in df.itertuples(index=True):
    print(bordered(row.state))
    # print(row)
    values = []
    to_get_dict = {}
    for col in row._fields:
        if 'metric' not in col:
            continue
        val = getattr(row, col)
        match = re.search(r'https://(.+)\.[govcomorgedus]{2,3}/', val)
        try:
            new_val = match.group(1)
        except:
            print(col,val)
            break
        to_get_dict[col] = new_val
        values.append(new_val)
        # print(f"{col}: {getattr(row, col)}")

    cnt = Counter(values)
    for char, count in cnt.items():
        print(bordered(f'{char}: {count}'))
        for k,v in to_get_dict.items():
            if char==v:
                print(k)

    print(to_get_dict.get(char))

    statess_dicts[row.state_abv] = to_get_dict

In [ ]:
Pretty

In [ ]:
al_data = pd.read_csv('demographics_2024_al.csv')
al_data

In [ ]:
import pprint
for k,v in statess_dicts.items():
    print('______________')
    print(k)
    pprint.pprint(v)